## 2D Bin Packing Problem

Dado un set de  𝑛  pallets (en este caso, 20) con un peso  𝑊𝑗  y altura  $H$𝑗  diferente por cada item  𝑗  =0,…,𝑛−1  y un número ilimitado de contenedores con un peso  $w_𝑦$$_𝑖$  y una altura  $h_y$$_i$  idénticos (todos los parámetros son enteros positivos), asignar a cada pallet un contenedor de manera que el peso y altura total de los pallets en cada contenedor no exceda las dimensiones del mismo donde, finalmente, el número de contenedores usados sea el mínimo.

## Definiciones

$n$ = Número de pallets\
$instance$_$name$ = Nombre de instancia\
$cw$ = Container Weight (Peso del contenedor)\
$ch$ = Container Height (Altura del contenedor)\
$pw$ = Pallet Weight (Peso del pallet)\
$ph$ = Pallet Height (Altura de pallet)

In [3]:
# Librerías
import pandas as pd
import numpy as np
from gurobipy import *

# Dataset
bpp_data_set = [
    # Muestra pequeña
    {
        'instance_name': 'first_test',
        'cw': 150,
        'ch': 100,
        'pw': [50, 50, 50],
        'ph': [50, 50, 50]
    },
    # Standard benchmark instance
    {
        'instance_name': 'u2400_00',
        'cw': 2400,
        'ch': 1300,
        'pw': [500, 500, 225, 350, 225, 125, 475, 400, 375, 375],
        'ph': [225, 500, 325, 350, 325, 200, 200, 125, 300, 400]
    },
    {
        'instance_name': 'u233_00',
        'cw': 233,
        'ch': 220,
        'pw': [76, 43, 81, 33, 99, 70, 72, 66, 84, 32, 34, 67, 25, 27, 81, 39, 74, 41, 78, 77],
        'ph': [30, 25, 55, 28, 73, 48, 46, 31, 30, 25, 25, 62, 23, 26, 44, 38, 65, 36, 34, 46]
    },
];

# Construcción de dataframe
df = pd.DataFrame(bpp_data_set, columns=['instance_name', 'cw', 'ch', 'pw', 'ph'])
df['n'] = df['pw'].apply(len)
# Pesos máximos y mínimos (pallet)
df['pwmin'] = df['pw'].apply(min)
df['pwmax'] = df['pw'].apply(max)
# Alturas máximos y mínimos (ßpallet)
df['phmin'] = df['ph'].apply(min)
df['phmax'] = df['ph'].apply(max)

# Muestra dataframe
display(df)

,instance_name,cw,ch,pw,ph,n,pwmin,pwmax,phmin,phmax
0,first_test,150,100,"[50, 50, 50]","[50, 50, 50]",3,50,50,50,50
1,u2400_00,2400,1300,"[500, 500, 225, 350, 225, 125, 475, 400, 375, ...","[225, 500, 325, 350, 325, 200, 200, 125, 300, ...",10,125,500,125,500
2,u233_00,233,220,"[76, 43, 81, 33, 99, 70, 72, 66, 84, 32, 34, 6...","[30, 25, 55, 28, 73, 48, 46, 31, 30, 25, 25, 6...",20,25,99,23,73


In [6]:
# Toma la primera instancia del dataset
bpp_data = bpp_data_set[2]

# Extrae los datos
cw, pw, ch, ph = bpp_data['cw'], bpp_data['pw'], bpp_data['ch'], bpp_data['ph']
n = len(pw) # Número de pallets
UB = n # Límite superior

# Imprime los datos
print('peso_contenedor =', cw, '| alto_contenedor =', ch, '| n =', n)

# Genera modelo de optimización
model = Model()
model.params.LogToConsole = True
model.params.TimeLimit = 120 # seconds

# Variables de decisión
x = model.addVars(n, UB, vtype=GRB.BINARY)
y = model.addVars(UB, vtype=GRB.BINARY)

# Función objetivo
model.setObjective(quicksum(y[i] for i in range(n)), GRB.MINIMIZE) # Minimizar el número de contenedores

# Restricciones
model.addConstrs(quicksum(x[i,j] for j in range(n)) == 1 for i in range(n)) # Cada pallet en 1 contenedor
model.addConstrs(quicksum(pw[i] * x[i,j] for i in range(n)) <= cw * y[j] for j in range(UB)) # El peso total de pallets debe ser inferior al peso del contenedor
model.addConstrs(quicksum(ph[i] * x[i,j] for i in range(n)) <= ch * y[j] for j in range(UB)) # El alto total de pallets debe ser inferior al alto del contenedor

model.optimize()
print("Model found a solution with n_bins =", model.objVal)

peso_contenedor = 233 | alto_contenedor = 220 | n = 20
Set parameter TimeLimit to value 120
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 60 rows, 420 columns and 1240 nonzeros
Model fingerprint: 0xe3fb24d4
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 14.0000000
Presolve time: 0.00s
Presolved: 60 rows, 420 columns, 1240 nonzeros
Variable types: 0 continuous, 420 integer (420 binary)

Root relaxation: objective 5.145923e+00, 89 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.14592    0    6   14.00000    5.1